In [1]:
import pickle 
import warnings

import numpy as np
import pandas as pd
import source

warnings.filterwarnings("ignore")

In [2]:
num_days = 10
num_trading_days = 30
initial_price = 100.0
utility_lambda = 0.1
mu = 0.0
sigma = 0.2

env_params = dict(
    num_days=num_days,
    num_trading_days=num_trading_days,
    mu=mu,
    sigma=sigma,
    initial_price=initial_price,
    utility_lambda=utility_lambda,
)


In [3]:
seq_jumps = pickle.load(open("./data/hardware_seq_jumps.pkl", "rb"))

np.set_printoptions(precision=3, suppress=True)
df = pd.read_csv("params/runs.csv")

strike_vals = [0.95, 1.0, 1.05]
cost_eps_vals = [0.0, 0.002]
critic_udpate_vals = ["vanilla", "expected", "distributional"]

res_columns = (
    ["critic_update", "seed"]
    + ["utility", "bs_utility"]
    + [f"PnL_{i}" for i in range(seq_jumps.shape[0])]
)

for cost_eps in cost_eps_vals:
    for strike in strike_vals:
        res_df = pd.DataFrame(columns=res_columns)
        for critic_udpate in critic_udpate_vals:
            if critic_udpate == "vanilla":
                model = "vanilla"
            else:
                model = "actor_critic"
            df_sub = df[
                (df["strike"] == strike)
                & (df["cost_eps"] == cost_eps)
                & (df["critic_update"] == critic_udpate)
            ]
            hparams = dict(**env_params, strike=strike, cost_eps=cost_eps, model=model)
            test_step = source.agent.make_test(**hparams)
            seeds = df_sub["seed"].values
            for seed in seeds:
                prefix = (
                    f"{hparams['cost_eps']}_{hparams['strike']}_{critic_udpate}_{seed}"
                )
                best_params = pickle.load(open(f"params/{prefix}-best_params.pkl", "rb"))
                test_metrics, other = test_step(seq_jumps, best_params)
                res_dict = {"critic_update": critic_udpate, "seed": seed}
                res_dict["utility"] = float(test_metrics["utility"])
                res_dict["bs_utility"] = float(test_metrics["bs_utility"])
                for i in range(seq_jumps.shape[0]):
                    res_dict[f"PnL_{i}"] = float(other[f"PnL_{i}"])
                res_df = res_df.append(res_dict, ignore_index=True)
        res_df.to_csv(f"results/results_{strike}_{cost_eps}.csv")

In [ ]:
table_columns = ["black_scholes", "vanilla", "expected", "distributional"]
strike_vals = [0.95, 1.0, 1.05]
cost_eps_vals = [0.0, 0.002]
critic_udpate_vals = ["vanilla", "expected", "distributional"]

for cost_eps in cost_eps_vals:
    table_df = pd.DataFrame(columns=table_columns)
    for strike in strike_vals:
        res_df = pd.read_csv(f"results/results_{strike}_{cost_eps}.csv")
        tb_dict = {}
        bs = res_df["bs_utility"].values[0]
        tb_dict["black_scholes"] = float(f"{bs:.3f}")
        for critic_udpate in critic_udpate_vals:
            best = max(
                res_df[res_df["critic_update"] == critic_udpate]["utility"].values
            )
            tb_dict[critic_udpate] = float(f"{best:.3f}")
        table_df = table_df.append(tb_dict, ignore_index=True)
    table_df.to_csv(f"results/table_{cost_eps}.csv")
    print(table_df)


   black_scholes  vanilla  expected  distributional
0         -7.514   -7.224    -7.301          -7.385
1         -4.415   -4.064    -4.193          -3.875
2         -2.768   -2.794    -2.633          -2.571
   black_scholes  vanilla  expected  distributional
0         -7.996   -7.468    -7.517          -7.656
1         -4.836   -4.639    -4.668          -4.424
2         -3.090   -3.067    -2.845          -2.791
